## 『本次練習內容』
#### 使用Xception backbone做 Trnasfer Learning


## 『本次練習目的』
  #### 了解如何使用Transfer Learning
  #### 了解Transfer Learning的優點，可以觀察模型收斂速度

In [1]:
%tensorflow_version 1.15

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `1.15`. This will be interpreted as: `1.x`.


TensorFlow 1.x selected.


##### 可以自行嘗試多種架構

In [2]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
import keras
from keras.layers import Input
 
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder
print(tf.__version__)

input_tensor = Input(shape=(32, 32, 3))
#include top 決定要不要加入 fully Connected Layer

'''Xception 架構'''

'''Resnet 50 架構'''
#model=keras.applications.ResNet50(include_top=False, weights='imagenet',
                                    #input_tensor=input_tensor,
                                    #pooling=None, classes=10)
model=keras.applications.ResNet50(input_shape=(32,32,3), include_top=False, weights='imagenet',pooling=None, classes=10)
model.summary()


Using TensorFlow backend.


1.15.2
Instructions for updating:
If using Keras pass *_constraint arguments to layers.



/tensorflow-1.15.2/python3.6/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 38, 38, 3)    0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 16, 16, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 16, 16, 64)   256         conv1[0][0]                      
___________________________________________________________________________________________

## 添加層數

In [3]:
x = model.output
x = GlobalAveragePooling2D()(x)
x = Dense(output_dim=128, activation='relu')(x)
x=Dropout(p=0.1)(x)
predictions = Dense(output_dim=10,activation='softmax')(x)
model = Model(inputs=model.input, outputs=predictions)
print('Model深度：', len(model.layers))

Model深度： 179


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=128)`
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.1)`
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=10)`
  """


## 鎖定特定幾層不要更新權重

In [4]:
for layer in model.layers[:100]:
    layer.trainable = False
for layer in model.layers[100:]:
    layer.trainable = True

## 準備 Cifar 10 資料

In [5]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7)
        return X_train, X_test
    
    
## Normalize Training and Testset    
x_train, x_test = normalize(x_train, x_test) 

## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder()
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()

170500096/170498071 [==============================] - 6s 0us/step
(50000, 32, 32, 3)


## Training

In [6]:
# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
model.fit(x_train,y_train,batch_size=32,epochs=100)


Epoch 1/100
50000/50000 [==============================] - 83s 2ms/step - loss: 1.2448 - accuracy: 0.5816
Epoch 2/100
50000/50000 [==============================] - 71s 1ms/step - loss: 1.0041 - accuracy: 0.6540
Epoch 3/100
50000/50000 [==============================] - 71s 1ms/step - loss: 0.8737 - accuracy: 0.6978
Epoch 4/100
50000/50000 [==============================] - 70s 1ms/step - loss: 0.7815 - accuracy: 0.7372
Epoch 5/100
50000/50000 [==============================] - 71s 1ms/step - loss: 0.7490 - accuracy: 0.7393
Epoch 6/100
50000/50000 [==============================] - 70s 1ms/step - loss: 0.6539 - accuracy: 0.7780
Epoch 7/100
50000/50000 [==============================] - 70s 1ms/step - loss: 0.5981 - accuracy: 0.7933
Epoch 8/100
50000/50000 [==============================] - 70s 1ms/step - loss: 0.5042 - accuracy: 0.8187
Epoch 9/100
50000/50000 [==============================] - 71s 1ms/step - loss: 0.4993 - accuracy: 0.8218
Epoch 10/100
50000/50000 [===================